<a href="https://colab.research.google.com/github/hxri/stdbias/blob/main/bias_in_colored_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import keras
import tensorflow as tf
from tqdm import tqdm
from time import sleep
import numpy as np
import math

In [28]:
def prepare_data():
  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_images = tf.image.grayscale_to_rgb(tf.expand_dims(train_images, axis=3)).numpy()
  test_images = tf.image.grayscale_to_rgb(tf.expand_dims(test_images, axis=3)).numpy()

  return train_images, test_images, train_labels, test_labels

In [23]:
def colorize(train_images, train_labels):
  cl = []
  t = []
  m = np.random.randint(7, size=(11, 1), dtype=np.uint8)
  m = np.unpackbits(m, axis=1)[:, -3:]
  m[:,-1] = 1
  m

  for u in range(len(np.unique(train_labels))):
    y = np.where(train_labels == u)
    t.append((math.floor(0.8 * len(y))))
    cl.append(y)

  for i in range(len(cl)):
    r = 0
    for j in cl[i]:
      if(r<=t[i]):
        train_images[j] = m[i] * train_images[j]
      else:
        train_images[j] = m[i+1] * train_images[j]
      r = r+1

In [24]:
def gen_model():
  model = keras.Sequential([
      tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),    
      tf.keras.layers.Conv2D(28, (3,3), activation='relu', input_shape=(28, 28, 3)),  
      tf.keras.layers.MaxPooling2D((2, 2)), 
      tf.keras.layers.Conv2D(56, (3,3), activation='relu'),  
      tf.keras.layers.Flatten(), 
      tf.keras.layers.Dense(64, activation='relu'),  
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [25]:
# Train and Evaluate on model
def train_and_evaluate(train_images, test_images, train_labels, test_labels, epochs):
  model = gen_model()
  history = model.fit(train_images, train_labels, epochs=epochs, verbose=0)
  accuracy = history.history['accuracy'][np.argmin(history.history['loss'])]
  print("Train accuracy : ", accuracy)
  eval = model.evaluate(test_images, test_labels, verbose=0)
  print("Test accuracy : ", eval[1])

In [26]:
def experiment(epochs=10):
  train_gray, test_gray, train_label_gray, test_label_gray = prepare_data()
  train_colored, test_colored, train_label_colored, test_label_colored = prepare_data()
  colorize(train_colored, train_label_colored)
  colorize(test_colored, test_label_colored)
  print("\n\nResults for Grayscale\n")
  train_and_evaluate(train_gray, test_gray, train_label_gray, test_label_gray, epochs)
  print("\n\nResults for Colored\n")
  train_and_evaluate(train_colored, test_colored, train_label_colored, test_label_colored, epochs)

In [29]:
experiment(epochs=5)

11501568/11490434 [==============================] - 0s 0us/step


Results for Grayscale

Train accuracy :  0.9953833222389221
Test accuracy :  0.9882000088691711


Results for Colored

Train accuracy :  0.9986500144004822
Test accuracy :  0.3434999883174896
